In [1]:
import numpy as np
import pandas as pd

from itertools import combinations
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('labeled.csv')
df.head()

,City,State,Population,Percent employed,Occupation (MBSA),Occupation (S),Occupation (SO),Occupation (RCM),Occupation (PT),Median household income,Homeownership rate,Median home price,Median rent,KMeans,Hierarchical
0,"Acres Green, Colorado",Colorado,2709.0,72.7,61.9,11.9,12.5,8.5,5.2,115536.0,77.4,540000.0,2663.0,0,3
1,"Aetna Estates, Colorado",Colorado,1177.0,72.4,12.5,27.3,31.8,17.0,11.4,63887.0,34.3,58200.0,1490.0,8,4
2,"Akron, Colorado",Colorado,1681.0,55.4,40.5,19.3,21.5,13.2,5.5,59470.0,70.4,186500.0,1070.0,5,0
3,"Alamosa, Colorado",Colorado,9847.0,61.8,34.7,20.7,26.5,6.6,11.4,46969.0,43.8,200200.0,881.0,8,4
4,"Alamosa East, Colorado",Colorado,1404.0,68.4,34.9,29.7,17.9,9.1,8.4,44741.0,68.5,219100.0,1100.0,3,4


In [3]:
df.shape

(1967, 15)

In [11]:
vectors = df.drop(columns=['City', 'State', 'KMeans', 'Hierarchical']).to_numpy()
vectors.shape

(1967, 11)

In [14]:
norms = np.linalg.norm(vectors, axis=1)
norms.shape

(1967,)

In [23]:
combos = combinations(range(0, 1966), 2)

In [24]:
cos_matrix = np.empty((1967, 1967))
for combo in combos:
    i = combo[0]
    j = combo[1]
    numerator = np.dot(vectors[i], vectors[j])
    denominator = norms[i] * norms[j]
    cos_sim = numerator / denominator
    cos_matrix[i,j] = cos_sim
    cos_matrix[j,i] = cos_sim  

In [26]:
np.argmax(cos_matrix, axis=1)

array([1182, 1635,  832, ...,  490,  184,    0], dtype=int64)

In [29]:
df.iloc[0]

City                       Acres Green, Colorado
State                                   Colorado
Population                                2709.0
Percent employed                            72.7
Occupation (MBSA)                           61.9
Occupation (S)                              11.9
Occupation (SO)                             12.5
Occupation (RCM)                             8.5
Occupation (PT)                              5.2
Median household income                 115536.0
Homeownership rate                          77.4
Median home price                       540000.0
Median rent                               2663.0
KMeans                                         0
Hierarchical                                   3
Name: 0, dtype: object

In [30]:
df.iloc[1182]

City                       Montgomery, Texas
State                                  Texas
Population                            2272.0
Percent employed                        62.0
Occupation (MBSA)                       42.3
Occupation (S)                          19.9
Occupation (SO)                         17.4
Occupation (RCM)                        14.0
Occupation (PT)                          6.4
Median household income              78333.0
Homeownership rate                      57.5
Median home price                   366600.0
Median rent                           1342.0
KMeans                                     7
Hierarchical                               4
Name: 1182, dtype: object